In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    import h5py

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import numpy as np
import pickle
from keras.models import model_from_json
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
MODEL_PATH = '../model/'
model_name = 'simple'
model_file = f'{model_name}.model.json'
weights_file = f'{model_name}.weights.h5'

def load_json_wih_weights_to_model():
    json_file = open(f'{MODEL_PATH}{model_name}/{model_file}', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(f'{MODEL_PATH}{model_name}/{weights_file}')
    print('Loaded \'%s\' model with weights from disk' % model_name)
    return loaded_model

In [3]:
# Load Tokenizer
with open(f'{MODEL_PATH}tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [4]:
# Load model
loaded_model = load_json_wih_weights_to_model()
loaded_model.summary()

Loaded 'simple' model with weights from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         2500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total params: 2,585,756
Trainable params: 2,585,756
Non-trainable params: 0
_____________________

In [5]:
classes_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def prepare_manual_comment(comment):
    tokenized_comment = tokenizer.texts_to_sequences(comment)
    return pad_sequences(tokenized_comment, maxlen=maxlen)

In [6]:
app = Flask(__name__)
CORS(app)
@app.route('/detox/api/comment', methods=['POST'])
def predict():
    payload = request.json['comment']
    
    # Tokenize the user's query and make a prediction
    prediction = loaded_model.predict([prepare_manual_comment([payload])], batch_size=1024)

    output = {}
    i = 0
    # Assign scores to classes
    for label in classes_names:
        output[label] = single_prediction[0][i]
        i += 1
        
    return output

# running REST interface, port=5000 for direct test
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0', port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
